In [1]:
# Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Input

Loading the data:

In [2]:
ts_data = pd.read_csv('/Users/kashyapava/Desktop/UIUC/SEM_4/STAT 430/Project 2/proj2_data_clean.csv',index_col='date', parse_dates=True)
ts_data.head(5)

,time,DOY,SWin,SWout,LWin,LWout,Rr,Rn,RH,PPFD,...,(z-d)/L,T*,v_var,w/co2_cov,w/h2o_cov,w/ch4_cov,FCH4,Ta,Tc,datetime
date,,,,,,,,,,,,,,,,,,,,,
2023-05-11,15:00,131.6249,695.4,135.20,398.9,507.4,0.194,451.60,35.8238,1340.0,...,-0.104409,-0.343706,1.337320,-0.004358,2.03306,-0.000042,-0.002199,300.282,308.48,2023-05-11 15:00:00
2023-05-11,15:30,131.6457,506.8,96.70,392.3,497.4,0.187,305.10,35.3730,982.0,...,-0.170200,-0.443938,1.958010,-0.006526,2.25563,-0.000043,0.002734,300.194,306.58,2023-05-11 15:30:00
2023-05-11,16:00,131.6665,319.4,60.97,395.8,477.3,0.190,177.00,35.6585,622.6,...,-0.099141,-0.290778,2.165070,-0.004225,1.91322,-0.000033,-0.000904,299.926,303.04,2023-05-11 16:00:00
2023-05-11,16:30,131.6873,338.2,64.57,382.2,474.9,0.188,181.00,35.4072,632.8,...,-0.060258,-0.178902,1.145780,-0.001495,1.65756,-0.000024,-0.002743,300.065,302.43,2023-05-11 16:30:00
2023-05-11,17:00,131.7082,121.5,22.22,377.4,456.5,0.184,20.15,36.9236,244.3,...,-0.060537,-0.161396,0.638123,-0.001278,1.20892,-0.000020,-0.002318,299.410,299.08,2023-05-11 17:00:00


In [3]:
# Exclude non-numeric predictors
ts_data_numeric = ts_data.select_dtypes(include=['float64', 'int64'])
ts_data_numeric.head(5)

,DOY,SWin,SWout,LWin,LWout,Rr,Rn,RH,PPFD,PPFDr,...,ET,(z-d)/L,T*,v_var,w/co2_cov,w/h2o_cov,w/ch4_cov,FCH4,Ta,Tc
date,,,,,,,,,,,,,,,,,,,,,
2023-05-11,131.6249,695.4,135.20,398.9,507.4,0.194,451.60,35.8238,1340.0,160.80,...,0.159395,-0.104409,-0.343706,1.337320,-0.004358,2.03306,-0.000042,-0.002199,300.282,308.48
2023-05-11,131.6457,506.8,96.70,392.3,497.4,0.187,305.10,35.3730,982.0,118.10,...,0.178391,-0.170200,-0.443938,1.958010,-0.006526,2.25563,-0.000043,0.002734,300.194,306.58
2023-05-11,131.6665,319.4,60.97,395.8,477.3,0.190,177.00,35.6585,622.6,74.94,...,0.147962,-0.099141,-0.290778,2.165070,-0.004225,1.91322,-0.000033,-0.000904,299.926,303.04
2023-05-11,131.6873,338.2,64.57,382.2,474.9,0.188,181.00,35.4072,632.8,76.32,...,0.125538,-0.060258,-0.178902,1.145780,-0.001495,1.65756,-0.000024,-0.002743,300.065,302.43
2023-05-11,131.7082,121.5,22.22,377.4,456.5,0.184,20.15,36.9236,244.3,29.03,...,0.093371,-0.060537,-0.161396,0.638123,-0.001278,1.20892,-0.000020,-0.002318,299.410,299.08


Transforming the data:

In [4]:
# Split into features and target
X = ts_data_numeric.drop(columns=['FCH4'])
y = ts_data_numeric['FCH4']

# Scaling data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

Train Test Split:

In [5]:
# Chronological Splitting
train_size = int(len(X_scaled) * 0.7)
val_size = int(len(X_scaled) * 0.2)

X_train, X_val, X_test = X_scaled[:train_size], X_scaled[train_size:train_size+val_size], X_scaled[train_size+val_size:]
y_train, y_val, y_test = y_scaled[:train_size], y_scaled[train_size:train_size+val_size], y_scaled[train_size+val_size:]

In [6]:
# Function to evaluate model
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'RMSE': rmse, 'MSE': mse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

1-step ahead Forecasting using FC, RNN, and LSTM:

In [44]:
# Fully Connected Feed Forward Networks
# Model 1: Single hidden layer

# Fully Connected Networks
model_fc1 = Sequential([Input(shape=(X_train.shape[1],)), Dense(26, activation='relu'), Dense(1)])
model_fc1.compile(optimizer='adam', loss='mse')
model_fc1.fit(X_train, y_train, epochs=30, batch_size=13, validation_data=(X_val, y_val), verbose=0)

model_fc2 = Sequential([Input(shape=(X_train.shape[1],)), Dense(52, activation='relu'), Dense(26, activation='relu'), Dense(1)])
model_fc2.compile(optimizer='adam', loss='mse')
model_fc2.fit(X_train, y_train, epochs=30, batch_size=13, validation_data=(X_val, y_val), verbose=0)

In [46]:
# Simple RNN
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_val_rnn = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

model_rnn1 = Sequential([Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])), SimpleRNN(26, activation='relu'), Dense(1)])
model_rnn1.compile(optimizer='adam', loss='mse')
model_rnn1.fit(X_train_rnn, y_train, epochs=30, validation_data=(X_val_rnn, y_val), verbose=0)

model_rnn2 = Sequential([Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])), SimpleRNN(52, activation='relu'), Dense(26, activation='relu'), Dense(1)])
model_rnn2.compile(optimizer='adam', loss='mse')
model_rnn2.fit(X_train_rnn, y_train, epochs=30, validation_data=(X_val_rnn, y_val), verbose=0)

In [47]:
# LSTM Models

# LSTM Model 1
model_lstm1 = Sequential([Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])), LSTM(26, activation='relu'), Dense(1)])
model_lstm1.compile(optimizer='adam', loss='mse')
model_lstm1.fit(X_train_rnn, y_train, epochs=30, validation_data=(X_val_rnn, y_val), verbose=0)

# LSTM Model 2
model_lstm2 = Sequential([Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])), LSTM(52, activation='relu'), Dense(26, activation='relu'), Dense(1)])
model_lstm2.compile(optimizer='adam', loss='mse')
model_lstm2.fit(X_train_rnn, y_train, epochs=30, validation_data=(X_val_rnn, y_val), verbose=0)

Results of 1 step forecasting:

In [48]:
# Predictions and Evaluations
results_1step = []
for name, model, data in zip(
    ['FC1', 'FC2', 'RNN1', 'RNN2', 'LSTM1', 'LSTM2'],
    [model_fc1, model_fc2, model_rnn1, model_rnn2, model_lstm1, model_lstm2],
    [X_test, X_test, X_test_rnn, X_test_rnn, X_test_rnn, X_test_rnn]
):
    y_pred = model.predict(data)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    y_test_inv = scaler_y.inverse_transform(y_test)
    metrics = evaluate_model(y_test_inv, y_pred_inv)
    metrics['Model'] = name
    results_1step.append(metrics)

results_1step_df = pd.DataFrame(results_1step)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [49]:
# Results
print('1-step Forecasting Results: ', '\n',results_1step_df)

1-step Forecasting Results:  
        RMSE       MSE       MAE       MAPE        R2  Model
0  0.023194  0.000538  0.017347  14.820231  0.315923    FC1
1  0.018538  0.000344  0.014155  33.177009  0.563007    FC2
2  0.052967  0.002806  0.039490  48.491303 -2.567633   RNN1
3  0.029765  0.000886  0.018144  13.865378 -0.126649   RNN2
4  0.029219  0.000854  0.018078  17.806238 -0.085673  LSTM1
5  0.027299  0.000745  0.023026  39.415203  0.052334  LSTM2


Overall, the LSTM model performed better than the fully connected network and the recurrent neural network.

The RNN (Recurrent Neural Network) performed better than the FC (Fully Connected network) which can imply that the recurrent connection enables it to better capture the temporal dependencies of the data as desired from the architectural intuition of the RNN.

The LSTM (Long Short Term Memory) performed better than the RNN (Recurrent Neural Network) which can imply that the forget gate is able to exclude the unnecessary information enabling it to better capture the temporal dependency by there by solving the issue of the vanishing/ exploding gradients in RNN over a long period of time.

Note that these inferences can be viable as the same architectural parameters (number of units) were used for comparing these models.

3-step ahead forecasting with RNN and LSTM:

In [33]:
# Corrected 3-step ahead forecasting setup
steps_ahead = 3
X_multi, y_multi = [], []
for i in range(len(X_scaled) - 2*steps_ahead):
    X_multi.append(X_scaled[i:i+steps_ahead, :])
    y_multi.append(y_scaled[i+steps_ahead:i+2*steps_ahead, 0])

X_multi, y_multi = np.array(X_multi), np.array(y_multi)

train_size_multi = int(len(X_multi)*0.7)
val_size_multi = int(len(X_multi)*0.2)

X_train_m, X_val_m, X_test_m = X_multi[:train_size_multi], X_multi[train_size_multi:train_size_multi+val_size_multi], X_multi[train_size_multi+val_size_multi:]
y_train_m, y_val_m, y_test_m = y_multi[:train_size_multi], y_multi[train_size_multi:train_size_multi+val_size_multi], y_multi[train_size_multi+val_size_multi:]

In [34]:
# RNN Model for 3-step ahead
model_rnn3 = Sequential([
    Input(shape=(X_train_m.shape[1], X_train_m.shape[2])),
    SimpleRNN(128, activation='relu', return_sequences=True),
    SimpleRNN(64, activation='relu'),
    Dense(steps_ahead)
])
model_rnn3.compile(optimizer='adam', loss='mse')
model_rnn3.fit(X_train_m, y_train_m, epochs=30, validation_data=(X_val_m, y_val_m), verbose=0)

In [35]:
# LSTM Model for 3-step ahead
model_lstm3 = Sequential([
    Input(shape=(X_train_m.shape[1], X_train_m.shape[2])),
    LSTM(128, activation='relu', return_sequences=True),
    LSTM(64, activation='relu'),
    Dense(steps_ahead)
])
model_lstm3.compile(optimizer='adam', loss='mse')
model_lstm3.fit(X_train_m, y_train_m, epochs=30, validation_data=(X_val_m, y_val_m), verbose=0)

Results for 3 step prediction:

In [36]:
# Evaluate and Display Results
results_3step = []
for name, model in zip(['RNN_3step', 'LSTM_3step'], [model_rnn3, model_lstm3]):
    y_pred = model.predict(X_test_m)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    y_test_inv = scaler_y.inverse_transform(y_test_m)
    metrics = evaluate_model(y_test_inv.flatten(), y_pred_inv.flatten())
    metrics['Model'] = name
    results_3step.append(metrics)

results_3step_df = pd.DataFrame(results_3step)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [37]:
# Results
print('3-step Forecasting Results:', '\n',results_3step_df)

3-step Forecasting Results: 
        RMSE       MSE       MAE       MAPE        R2       Model
0  0.060318  0.003638  0.046163  74.166938 -3.564378   RNN_3step
1  0.036506  0.001333  0.025350  33.345497 -0.671908  LSTM_3step


Overall, the LSTM model performed better than the recurrent neural network in 3-step forecasting.

The LSTM (Long Short Term Memory) performed better than the RNN (Recurrent Neural Network) which can imply that the forget gate is able to exclude the unnecessary information enabling it to better capture the temporal dependency by there by solving the issue of the vanishing/ exploding gradients in RNN over a long period of time.

Note that these inferences can be viable as the same architectural parameters (number of units) were used for comparing these models.